# CSCI 3022 Homework
<figure>
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right">
</figure>

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below. 

In [60]:
NAME = "Tyler Cranmer"
COLLABORATORS = ""

If you referenced any web sites or solutions not of your own creation, list those references here:

* List any external references or resources here

# Homework 4

In this homework, you'll practice material from the Proability portion of the course, covering

* Joint distributions, covariance and correlation

In [61]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats as stats
import math

# Problem #1 - Joint Random Variables

In this problem we will construction a *joint distribution* from the titantic data that attempts to capture the probability of survival as functions of the passenger class and sex of the passengers.

In [84]:
ti = pd.read_csv('https://raw.githubusercontent.com/jorisvandenbossche/pandas-tutorial/master/data/titanic.csv')

In [85]:
ti.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## (1a)

You should produce a Pandas DataFrame named `ti_jrv` ("titanic joint randon variable) similar to the fragment shown below:

| Index	| Pclass |	Sex	| Prob |
|-----|-------|------|-------- |
|0 |	1 |	1	| 0.131579 |
|1 |	2 |	2	| 0.204678 |

Your table should include the product of passenger class (1,2,3) and Sex (1, 2). The rows can be in any order.

The first row in this example can be interpreted that 13.15% of those survived are male passengers in the first class while 20.5% of the survived are female passengers in the second class. 

Your table will contain a total of 6 rows. You can either construct the table in columns and then create the DataFrame or create an empty DataFrame and then [add rows to the frame](https://stackoverflow.com/questions/17091769/python-pandas-fill-a-dataframe-row-by-row) using `loc` or a method of your choosing.

In [212]:
df = pd.DataFrame(ti)

total_survived = [] #array to hold all the people that survived
for item in df.Survived:
    if item == 1:
        track_num = item
        total_survived.append(track_num)

#replaced famle and male with values 1 and 2        
df['Sex'].replace('female', 2, inplace=True)
df['Sex'].replace('male',1,inplace=True)
df['Sex'].astype('int64', copy=False)

#make new dataframe that is grouped by Pclass and Sex. Aggregate the sum of the Survived data. 
ti_jrv = df.groupby(['Pclass', 'Sex']).agg({'Survived' : ['sum']})
ti_jrv.columns = ['Prob']
ti_jrv['Prob'] = ti_jrv.Prob.apply(lambda x : x / len(total_survived)) #find the probability of earch row.
ti_jrv = ti_jrv.reset_index()
ti_jrv


,Pclass,Sex,Prob
0,1,1,0.131579
1,1,2,0.266082
2,2,1,0.049708
3,2,2,0.204678
4,3,1,0.137427
5,3,2,0.210526


### (1b) Argue that this is isn't a valid probability distribution

Argue that this is or isn't a valid probability distribution over the product space of `Pclass * Sex`.

Although this shows a valid probability distribution over Pclass * Sex because the all probabilities add up to 1, we did alter the data origional data to show the probability of a class and sex surving out of just the individuals who survived. We removed the individuals who did not survive, ultimately changing the probility distribution.


### (1c)  "Marginalize" the probability distribution to compute the survival probability by Sex alone

Using the definition of marginalization, indicate the probability distribution of survival by Sex alone. 

You should assign your results to the variable `ti_sx` and print out your results.

In [213]:
# your code here
# create new dataframe with the marginalize values of each sex's survival probability 
ti_sx = ti_jrv.groupby(['Sex']).agg({'Prob': 'sum'})
ti_sx = ti_sx.reset_index()
ti_sx

,Sex,Prob
0,1,0.318713
1,2,0.681287


### (1d) Computed the expectation of the survival over the the Sex R.V.

Assign your answer to the variable `ti_sx_e`

In [214]:
# your code here
# E[Sex]
ti_sx_e = (ti_sx['Sex']*ti_sx['Prob']).sum()
ti_sx_e

1.6812865497076024

### (1e) Marganlize the probability to compute survival probability for Pclass alone

Using the definition of marginalization, indicate the probability distribution of survival by Pclass alone. 

You should assign your results to the variable `ti_cl` and print out your results.

In [215]:
# your code here
# create new dataframe with the marginalize values of each Pclass's survival probability 
ti_cl = df.groupby(['Pclass']).agg({'Survived' : ['sum']})
ti_cl.columns = ['Prob']
ti_cl['Prob'] = ti_cl.Prob.apply(lambda x : x / len(total_survived))
ti_cl = ti_cl.reset_index()

ti_cl


,Pclass,Prob
0,1,0.397661
1,2,0.254386
2,3,0.347953


### (1f) Compute the expectation of survival over the Pclass R.V.

Assign your answer to the variable `ti_cl_e`

In [216]:
# your code here
# E[Pclass]
ti_cl_e = (ti_cl['Pclass']*ti_cl['Prob']).sum()
ti_cl_e

1.9502923976608186

### (1g) Compute the joint expectation of survival E[Sex Pclass]

Assign your answer to the variable `ti_jrv_e`

In [217]:
# your code here
# E[XY]
ti_jrv_e = (ti_jrv['Pclass'] * ti_jrv['Sex'] * ti_jrv['Prob']).sum()
ti_jrv_e

3.2573099415204676

### (1h) Compute the Covariance

Compute the covariance of the Sex Pclass joint distribution. Assign the value to `ti_jrv_cov` and print out the value. You can use any method discussed in Chapter 10.2 to compute the covariance although some will be easier than others.

In [218]:
# your code here
# Cov(X,Y) = E[XY] - E[X]E[Y]
ti_jrv_cov = ti_jrv_e - (ti_cl_e*ti_sx_e)
ti_jrv_cov

-0.0216904346636575

### (1i) Argue that R.V.'s Sex and PClass are or are not independent. 

Based on the computations above, argue in words why you do or do not believe that the surival rate distributed by Sex and PClass are independent. Use quantitative arguments if possible.

Based on the data from the Covariance, the surival rate distributed by Sex and PClass are dependent but negatively correlated. If the RV were independent of eachother then E[XY] = E[X]E[Y] and the Cov(X,Y) would equal 0.


A negative value means the survival rate increases when the class goes down (i.e towards 1st class) and the gender goes up (male = 1 , female =2), showing opposite behavior 

### (1j) Calculate the correlation coefficient of the Sex and PClass variables.

You should assign the final value to variable `ti_jrv_cor`

In [219]:
#your code here
#Variance
#Var(X) = E[X^2] - (E[X])^2
E_sexSqr = ((ti_sx['Sex']*ti_sx['Prob'])**2).sum()
var_sex = E_sexSqr - (ti_sx_e**2)

#Var(Y) = E[Y^2] - (E[Y])^2
E_pclassSqr = ((ti_cl['Pclass']*ti_cl['Prob'])**2).sum()
var_pclass = E_pclassSqr - (ti_cl_e**2)

#correlation coefficient
#p(X,Y) = Cov(X,Y) / (sqrt(Var(X)*Var(Y)))
ti_jrv_cor = ti_jrv_cov / math.sqrt(var_sex*var_pclass)
ti_jrv_cor

-0.015356466294964283

### (1k) Are the survival probabilities between Sex and PClass correlated?

Use the data from the joint probability distribution to argue why your correlation coefficient makes sense. You can also use the probability distribution of the marginal R.V.'s (i.e. just Sex or just PClass) to motivate your argument, but you must eventually explain why the *joint* distribution has the correlation coefficient you've calculated.

The Correlation Coefficient being negative is consistant with the covariance being negative as well. The covariance shows us that the two varables are moving away from eachother and the Correlation Coefficient backs that up because the value is between 0 and -1. 

There is a slight negative correlation of the survival probability between Sex and Pclass. This makes sense because you can see that from the marginal data for the Pclass probability goes 1st class .39 , 2nd calss .25 and 3rd class .34. Which shows that the probability of survival goes down then up but still lower than first class. Which shoes a negative downward trend. For the marginal data of Sex, the probability goes males(1) = .31 and females(2) = .68. Which shows an upward trend. Ultimately giving the correlation coefficient a negative value. Since the negative value is closer to zero, this also tells us that there isnt always a consistant decrease in probabilities of one of the variables. 

The big picture reasoning be the survival rate increases when the class goes down (i.e towards 1st class) and the gender goes up (male = 1 , female =2), showing opposite behavior

In [2]:
temp = cdf_binomial(1,3,0.07)

NameError: name 'cdf_binomial' is not defined